In [1]:
#| default_exp core

In [2]:
#|export
from __future__ import annotations
from IPython.display import Markdown, display
import os, re
from dataclasses import dataclass
from typing import Union, Optional, Any, Collection, Iterable, TypeVar

from nbdev.export import nb_export

# V0 Plans

Class for things that can be used in math text - as exponents, subscripts etc

Class for environments - align, equation, etc

Use IPython display to render

# References

[Emulating numeric types docs](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types)

# Code

In [3]:
#|export
def to_node(x: TNS) -> TexNode:
    """
    Converts a raw string to a TexAtom
    Returns a TexNode unaltered
    """
    if isinstance(x, str):
        return TexAtom(x)
    else:
        return x

class TexBase:
    @property
    def tex(self) -> str:
        raise NotImplementedError("Children should implement this")

    def display(self) -> None:
        display(Markdown("$$" + self.tex + "$$"))

class TexNode(TexBase):
    # Methods to mimic arithmetic

    # Use self + other
    def __add__(self, other: TexNode) -> TexSequence:
        return TexSequence((self, to_node(other)))

    # Use self ** other
    def __pow__(self, other: TexNode) -> TexPow:
        return TexPow(self, other)

    # Use self / other
    def __truediv__(self, other: TexNode) -> TexFrac:
        return TexFrac(self, other)

    
class TexAtom(TexNode):
    s: str
    def __init__(self, s: str) -> None:
        super().__init__()
        self.s = s
    
    @property
    def tex(self) -> str:
        return self.s


TNS = TypeVar("TNS", bound=str|TexNode)

class TexSequence(TexNode):
    children: tuple[TexNode, ...]
    def __init__(self, children: Collection[TexNode]) -> None:
        super().__init__()
        self.children = tuple(children)
    
    @property
    def tex(self) -> str:
        return " ".join([c.tex for c in self.children])

class TexPow(TexNode):
    base: TexNode
    exp: TexNode
    def __init__(self, base: TNS, exp: TNS) -> None:
        super().__init__()
        self.base = to_node(base)
        self.exp = to_node(exp)
    
    @property
    def tex(self) -> str:
        return self.base.tex + "^{" + self.exp.tex + "}"

class TexFrac(TexNode):
    def __init__(self, num: TNS, den: TNS) -> None:
        super().__init__()
        self.num = to_node(num)
        self.den = to_node(den)

    @property
    def tex(self) -> str:
        return "\\frac{" + self.num.tex + "}" + "{" + self.den.tex + "}"

class TexColored(TexNode):
    child: TexNode
    color: str
    def __init__(self, child: TNS, color: str) -> None:
        super().__init__()
        child = to_node(child)
        self.child = child
        self.color = color
    
    @property
    def tex(self) -> str:
        return "{\\color{" + self.color + "}" + self.child.tex + "}"
    

In [4]:
#|export
def jointex(sep: str,  children: Collection[TexNode]) -> TexAtom:
    s = ""
    for i, c in enumerate(children):
        s += c.tex
        if i < len(children) - 1:
            s += sep
    return TexAtom(s)

In [5]:
#|export
class TexEnvironment(TexBase):
    def __init__(self, nm: str) -> None:
        super().__init__()
        self.nm = nm
    
    @property
    def begin_tex(self) -> str:
        return "\\begin{" + self.nm + "}"
    
    @property
    def end_tex(self) -> str:
        return "\\end{" + self.nm + "}"

class TexList(TexEnvironment):
    # Should nm instead be a class-level attribute?
    def __init__(self,nm: str, children: Collection[TNS]) -> None:
        super().__init__(nm)
        self.children  = tuple([to_node(c) for c in children])

    @property
    def tex(self) -> str:
        lines = jointex("\\\\\n", self.children)
        s = self.begin_tex + "\n" + lines.tex + "\n" + self.end_tex
        return s


# Should we have special consideration for alignment with & ?
class TexAlign(TexList):
    def __init__(self, children: Collection[TNS]) -> None:
        super().__init__("align", children)


In [6]:
x = TexColored("x", "blue")
y = TexColored("y", "red")
xy = x / y
xy.display()

$$\frac{{\color{blue}x}}{{\color{red}y}}$$

In [7]:
TexAlign([x,y]).display()

$$\begin{align}
{\color{blue}x}\\
{\color{red}y}
\end{align}$$

In [8]:
TexAlign(["x &= y^2", "y &= 2z + \\pi"]).display()

$$\begin{align}
x &= y^2\\
y &= 2z + \pi
\end{align}$$

### Bayes rule example

In [9]:
#|export

class Prob(TexNode):
    cond: Optional[TexNode]
    def __init__(self, event: Union[str, TexNode], cond: Optional[Union[str, TexNode]] = None) -> None:
        super().__init__()
        self.event = to_node(event)
        if cond is None:
            self.cond = None
        else:
            self.cond = to_node(cond)

    @property
    def tex(self) -> str:
        s = "\\mathrm{P}\\left( " + self.event.tex
        if self.cond is not None:
            s += "\\mid " + self.cond.tex
        s +=  " \\right)"
        return s

In [10]:
# Putting this in a function to avoid polluting global scope

def example_bayes_rule():
    a = TexColored("A", "red")
    b = TexColored("B", "blue")
    left = Prob(a, b)
    right_num = Prob(b, a) + Prob(a)
    right_den = Prob(b)
    right = right_num / right_den
    eq: TexNode = left + " = " + right
    eq.display()

example_bayes_rule()

$$\mathrm{P}\left( {\color{red}A}\mid {\color{blue}B} \right)  =  \frac{\mathrm{P}\left( {\color{blue}B}\mid {\color{red}A} \right) \mathrm{P}\left( {\color{red}A} \right)}{\mathrm{P}\left( {\color{blue}B} \right)}$$

$$
\begin{bmatrix}
1 & 2 & 3\\
a & b & c
\end{bmatrix}
$$

In [11]:
#|export
class TexMatrix(TexNode):
    def __init__(self, els: Collection[Collection[TNS]]) -> None:
        super().__init__()
        self.els = tuple([tuple([to_node(x) for x in row]) for row in els])
    @property
    def tex(self) -> str:
        s = "\\begin{bmatrix}\n"
        lines = [jointex(" & ", row) for row in self.els]
        s_content = jointex("\\\\\n", lines)
        s += s_content.tex
        s += "\n\\end{bmatrix}"
        return s

In [12]:
data = [["x"], ["y"]]
TexMatrix(data).display()
TexMatrix(
    [["x^0", "x^1"], ["y^0", TexColored("y","red") ** "1"]]
).display()

$$\begin{bmatrix}
x\\
y
\end{bmatrix}$$

$$\begin{bmatrix}
x^0 & x^1\\
y^0 & {\color{red}y}^{1}
\end{bmatrix}$$

In [13]:
#|export
class TexRoot(TexNode):
    def __init__(self, child: TNS, power: Optional[TNS] = None) -> None:
        super().__init__()
        self.child = to_node(child)
        if power is None:
            self.power = None
        else:
            self.power = to_node(power)
    @property
    def tex(self) -> str:
        s = "\\sqrt"
        if self.power is not None:
            s += "[" + self.power.tex + "]"
        s += "{" + self.child.tex + "}"
        return s

In [14]:
TexRoot("x").display()
TexRoot("x", "3").display()

$$\sqrt{x}$$

$$\sqrt[3]{x}$$

In [15]:
nb_export('latex_gen.ipynb', 'pytexgen')